---
title: "Ensembles"
theme: Montpellier
author: "Máster en Ciencias de Datos e Ingeniería de Computadores, Minería de Datos - Preprocesamiento y clasificación"
date: 11/11/2024
date-format: long
toc: true
toc-title: Tabla de Contenidos
toc-depth: 1
execute:
  echo: true
output:
  beamer_presentation:
    slide_level: 1
format:
  html:
    code-fold: false
    code-summary: "Muestra código"
    fig-width: 5
    fig-height: 3
    fig-align: left
  beamer:
    fig-width: 4
    fig-height: 2
  revealjs:
    theme: dark
    fig-align: left
    fig-height: 5
    fig-cap-location: margin
    smaller: true
---

## Ensembles

En este notebook vamos a repasar los principales métodos de Ensemble vistos en teoría. 

Veremos los _ensembles_ o modelos compuestos.

Se aplica una evaluación completamente *naif* de los clasificadores, sin ningún esquema de validación. Esta tarea se deja al estudiante para repasar los contenidos de la primera sesión de prácticas. 

In [ ]:
#| code-fold: true
import sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
#from sklearn.tree import DecisionTreeClassifier
from sklearn import datasets
from sklearn.datasets import load_breast_cancer
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score, classification_report

iris = datasets.load_iris()
X_i = iris.data
y_i = iris.target

breastCancer = datasets.load_breast_cancer()
X_b = breastCancer.data
y_b = breastCancer.target

# Voto Simple

## Combinación de Voto Simple

Son los más sencillos, se basan en la idea de tener varios modelos (normalmente distintos) para un problema de clasificación, y elegir la clase seleccionada por la mayoría de los modelos.

In [ ]:
from sklearn.ensemble import VotingClassifier
from  sklearn.model_selection import cross_val_score, cross_validate

log = LogisticRegression(multi_class='multinomial')
cart = tree.DecisionTreeClassifier( max_depth=5, criterion="gini")
naiveBayes = GaussianNB()

# VotingClassifier es un clasificador que toma como entrada una lista de clasificadores y un parámetro de votación.
# Vamos a probar primero con el voto por mayoría, con el parámetro 'hard'
ensembleVotoSimple = VotingClassifier(estimators=[
        ('logistic', log), ('cart', cart), ('NaiveBayes', naiveBayes)], voting='hard')

ensembleVotoSimple.fit(X_i, y_i)

y_pred = ensembleVotoSimple.predict(X_i)

print("Informe completo\n",classification_report(y_i, y_pred))

---

## Considerando la confianza

En Sklearn existe otra forma de combinar los votos: en lugar de utilizar la clase más votada, podemos emplear como salida la del clasificador que tenga más confianza. 

Se recomienda cuando se sabe que los clasificadores están bien ajustados.

In [ ]:
ensembleVotoSimple = VotingClassifier(estimators=[
        ('logistic', log), ('cart', cart), ('NaiveBayes', naiveBayes)], voting='soft')

#Utilizamos el método fit y predict sobre en ensemble
ensembleVotoSimple.fit(X_i, y_i)

y_pred = ensembleVotoSimple.predict(X_i)

print("Informe completo\n",classification_report(y_i, y_pred))

---

Aplicando validación cruzada:

In [ ]:
from sklearn.model_selection import KFold
estimators=[('logistic', log), ('cart', cart), ('NaiveBayes', naiveBayes)]

# VotingClassifier es un clasificador que toma como entrada una lista de clasificadores y un parámetro de votación.
# Vamos a probar primero con el voto por mayoría, con el parámetro 'hard'
ensembleVotoHard = VotingClassifier(estimators=estimators, voting='hard')
ensembleVotoSoft= VotingClassifier(estimators=estimators, voting='soft')

cv=KFold(n_splits=5, shuffle=True, random_state=53)

result1 = cross_val_score(ensembleVotoHard, X_i, y_i, cv=cv, scoring='accuracy')
print(result1)
result2 = cross_val_score(ensembleVotoSoft, X_i, y_i, cv=cv, scoring='accuracy')
print(result2)

---

## Ejercicios propuestos

1. Utilice diferentes clasificadores en el ensemble.
2. Es conocido que aumentar la variedad de los clasificadores también permite mejorar la precisión global del ensemble. Pruebe a incluir clasificadores diversos o que ajusten poco (lo que se conoce como "weak clasifiers").
3. Combine el uso de ensembles con un esquema de particionamiento correcto.

# Bagging

## Bagging

En el bagging  se dividen los datos, y se reparten entre los modelos. Por defecto se dividen por instancias, pero se puede dividir también por características.

In [ ]:
from sklearn.ensemble import BaggingClassifier

#Vamos a fijar el random_state para que los resultados sean reproducibles y podamos evaluar bien el efecto de cambiar el 
# clasificador y el número de estimadores
bagging = BaggingClassifier(estimator=tree.DecisionTreeClassifier(), n_estimators=10, random_state=0);

bagging.fit(X_i, y_i)

y_pred = bagging.predict(X_i)

print("Informe completo\n",classification_report(y_i, y_pred))

## Ejercicios Propuestos

1. Bagging depende mucho de que los clasificadores base sean capaces de ajustar bien en cada subconjunto: ante la selección de ejemplos y características en cada fase del boostrap, sería ideal que produjesen modelos que produzcan buen rendimiento. Pruebe con otros clasificadores para intentar mejorar la precisión del modelo (se recomienda al menos aplicar un Hold-out para esto).

2. El número de estimadores o fases de booststrap también tiene un impacto significativo, pero tiende a diluirse al aumentar su valor. Pruebe a intentar encontrar el menor número de fases en combinación con el clasificador más apropiado del punto anterior.

3. Pruebe a simplificar también las características usando el parámetro **bootstrap_features**.

## Random Forest

Es el modelo de Bagging más popular, en este caso se descompone principalmente por características.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=10, random_state=0)

rf.fit(X_i, y_i)

y_pred = rf.predict(X_i)

print("Informe completo\n",classification_report(y_i, y_pred))

## Ejercicios propuestos

1. Experimente con el parámetro **max_features**, que controla el número de características para hacer la división.
2. De igual manera, el parámetro **max_samples** controla el número de ejemplos a cada sub-árbol.
3. Principalmente, el rendimiento de RandomForest puede ajustarse rápidamente controlando el número de árboles y su profundidad. En este caso, interesa que los árboles sean diversos: deduzca qué valores de estos 2 parámetros favorecen esta propiedad.

# Boosting

## Boosting

- En el bosting, en vez de resolver en paralelo como en el Bagging, los nuevos modelos se centran en clasificar los mal clasificados de los anteriores.

Hay varios algoritmos populares:

- AdaBoost: Algoritmo original.
- XGboost: Algoritmo muy popular, mejor gestión de datos perdidos, uso de técnicas para evitar sobreaprendizaje, y GradientBoosting.
- CatBoost: Diferente gestión de ruido, y mejor soporte de atributos categóricos.
- LightGBM: Centrado en mejorar el rendimiento, no los resultados. 

## AdaBoost

Está directamente soportado:

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

# Si no se indica estimators, aplica árboles de decisión
adaboost = AdaBoostClassifier(n_estimators=10, random_state=0)

adaboost.fit(X_i, y_i)

y_pred = adaboost.predict(X_i)
print("Informe completo\n",classification_report(y_i, y_pred))

## Ejercicios propuestos

1. Si no se especifica el parámetro **base_estimator**, se utiliza DecisionTreeClassifier por defecto. En el caso de AdaBoost, es interesante tener clasificadores débiles que produzcan ajustes diversos a los datos. Pruebe otros clasificadores débiles, como LogisticRegression o 1NN (por ejemplo).

2. El learning rate controla cómo "sobreajustamos" a los datos que resultaron incorrectos en la iteración previa. Pruebe a jugar incrementando el learning rate con **learning_rate** frente al número de iteraciones **n_estimators**, intentando reducir al máximo el segundo argumento.

## Gradient Boosting

El GradientBoosting, a diferencia de Adaboost, ajusta en base al error (la función 'loss') del clasificador anterior, con lo que se puede mejorar/acelerar mucho el ajuste. 

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gradboost = GradientBoostingClassifier(n_estimators=10, random_state=0)

gradboost.fit(X_i, y_i)

y_pred = gradboost.predict(X_i)
print("Informe completo\n",classification_report(y_i, y_pred))

## Ejercicios propuestos

1. El parámetro **loss** representa la función de pérdida que se desea optimizar y tiene un gran impacto en el rendimiento. Pruebe los diferentes valores.

2. Estudie los parámetros **learning_rate** y **n_estimators** de la misma forma que se hizo con AdaBoost. Si mantenemos el número de iteraciones **n_estimators** igual que AdaBoost, ¿Qué algoritmo produce un mejor ajuste final?

## XGBoost

Vamos a instalar XGBoost en nuestro entorno

Si tienes Anaconda, podemos utilizar *conda install -c conda-forge xgboost*

En otro caso, *pip install xgboost* debería funcionar.

In [ ]:
#|output: false
!pip install xgboost

El API es igual que el resto de modelos.

Se recomienda visitar [https://xgboost.readthedocs.io/en/stable/python/python_api.html#xgboost.XGBClassifier ](https://xgboost.readthedocs.io/en/stable/python/python_api.html#xgboost.XGBClassifier) para una lista completa de los parámetros.

---

Vamos a aplicarlo sobre el problema.

In [ ]:
import xgboost as xgb

xgb = xgb.XGBClassifier()

xgb.fit(X_i, y_i)

y_pred = xgb.predict(X_i)
print("Informe completo\n",classification_report(y_i, y_pred))

## Ejercicios propuestos

XGBoost ofrece muy buen rendimiento con la configuración por defecto. Los parámetros **booster**, **learning_rate** y **n_estimators** son los que tienen un impacto más visible en el rendimiento. No obstante, XGBoost suele utilizarse con un ajuste/búsqueda de parámetros debido a su alto número, por lo que dejamos al estudiante que pruebe libremente en este apartado de ejercicios propuestos.

## CatBoost 

Es necesario instalarlo externamente, ya sea con *conda install -c conda-forge catboost* o *pip install catboost*.

In [ ]:
#|output: false
!pip install catboost

Más información sobre los parámetros en [https://catboost.ai/en/docs/concepts/python-reference_catboostclassifier](https://catboost.ai/en/docs/concepts/python-reference_catboostclassifier)

--- 

Ejemplo sobre el problema anterior.

In [ ]:
from catboost import CatBoostClassifier

catB = CatBoostClassifier(iterations=2, learning_rate=1, depth=2, loss_function='MultiClass')

catB.fit(X_i, y_i)

y_pred = catB.predict(X_i)
print("Informe completo\n",classification_report(y_i, y_pred))

--- 

Una de las ventajas de **CatBoost** es que es capaz de tratar con atributos con valores nominales. Vamos a ilustrarlo con el dataset monk2, un clásico con todos los atributos nominales.

In [ ]:
#| code-fold: true
from sklearn.datasets import fetch_openml

monk2 = fetch_openml(name='monks-problems-2', version=1);
X_k = monk2.data
y_k = monk2.target
print(X_k.head())
print(y_k.head())

catB = CatBoostClassifier(iterations=2, 
                          learning_rate=1, 
                          depth=2, 
                          cat_features=[0,1,2,3,4,5], #Indicamos que las columnas 0,1,2,3,4,5 son categóricas (todas las del conjunto)
                          loss_function='Logloss'); #Usamos Logloss porque es un problema de clasificación binaria

catB.fit(X_k, y_k)

y_pred = catB.predict(X_k)
print("Informe completo\n",classification_report(y_k, y_pred))

## Ejercicios propuestos

1. Con la configuración actual, se obtiene 0 aciertos para la clase 1. Ajuste los parámetros para evitarlo (es un problema muy desequilibrado).
2. Pruebe monk2 con otros algoritmos de clasificación para observar el comportamiento con las etiquetas categóricas (si es que se pueden ejecutar...)

## LightGBM

De nuevo, se trata de un clasificador reciente que Sklearn no ha incorporado. No obstante, la implementación oficial tiene una interfaz compatible con Sklearn.

Para instalarlo en conda: **conda install -c conda-forge lightgbm**

Para instalarlo con pip: **pip install lightgbm**

In [ ]:
#|output: false
!pip install lightgbm

La información sobre los parámetros se puede consultar en [https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMClassifier.html#lightgbm.LGBMClassifier](https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMClassifier.html#lightgbm.LGBMClassifier)

---

In [ ]:
from lightgbm import LGBMClassifier

lgbm = LGBMClassifier(objective='multiclass') #Indicamos que es un problema de clasificación multiclase

lgbm.fit(X_i, y_i)

y_pred = lgbm.predict(X_i)
print("Informe completo\n",classification_report(y_i, y_pred))

----

In [ ]:
from lightgbm import plot_split_value_histogram

#Vamos a mirar como se llaman las columnas que conoce LightGBM para pintar el histograma de división
lgbm.booster_.feature_name() 
plot_split_value_histogram(lgbm, feature='Column_0')

## Uso con one-hot

LightGBM también trata con atributos nominales de forma nativa usando One-hot encoding. Vamos a probarlo.

In [ ]:
lgbm = LGBMClassifier(objective='binary') #Indicamos que es un problema de clasificación binario

lgbm.fit(X_k, y_k)

y_pred = lgbm.predict(X_k)
print("Informe completo\n",classification_report(y_k, y_pred)) #Observa la diferencia con la clase 1 frente a CatBoost

----

Vamos a mirar cómo se llaman las columnas que conoce LightGBM para pintar el histograma de división

In [ ]:
lgbm.booster_.feature_name() #Vamos a mirar como se llaman las columnas que conoce LightGBM para pintar el histograma de división
# plot_split_value_histogram(lgbm, feature='attr1') # Aún no funciona para atributos categóricos :(

# Stacking

## Clasificadores Stacking

En este caso se aplica primero cada modelo, y luego otro modelo combina los resultados de éstos para dar la predicción definitiva.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import StackingClassifier

estimators = [
    ('dt', DecisionTreeClassifier(random_state=100)),
    ('svr', KNeighborsClassifier(n_neighbors=1)),
    ('mlp', MLPClassifier(random_state=100)) 
]
#El clasificador final es una regresión logística
stack = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression()) 

stack.fit(X_i, y_i)

y_pred = stack.predict(X_i)
print("Informe completo\n",classification_report(y_i, y_pred))

---

## Ejercicios propuestos

1. El uso de stacking acepta otros ensembles como clasificadores base. Pruébelos.
2. Aunque suele recomendarse un modelo simple como clasificador final (el que se entrena sobre las predicciones), puede probar otro clasificador más complejo con una validación adecuada para intentar obtener resultados mejores.

# Ejercicios Finales

1. Aplique los modelos de Votación para resolver el problema del cáncer, aplicando _Cross Validation_.

2. Aplique los distintos modelos de Boosting.

3. Aplique un par de modelos de Stacking: Usando LogisticRegression, y un MLP.